<a href="https://colab.research.google.com/github/cohmathonc/biosci670/blob/master/IntroductionComputationalMethods/exercises_solutions/07_ODEs_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pylab as plt

# Solving ODEs

You see the following plot (if not displayed, view [here](https://github.com/cohmathonc/biosci670/blob/master/IntroductionComputationalMethods/exercises/figs/ODE_oscillating_solution.png)) in a report:

<img src="https://github.com/cohmathonc/biosci670/blob/master/IntroductionComputationalMethods/exercises/figs/ODE_oscillating_solution.png?raw=1">

The authors state that the plot shows the simulated time evolution of a specific cell growth model, and they report an oscillatory growth pattern. They also mention that the explicit Euler method was used to solve the ODE that describes the growth model.

1. You suspect that the reported oscillatory pattern might be a numerical artifact. Why? 
   
   (Review *1.1.1 Euler Method (explicit)* in the [ODE notebook](https://github.com/cohmathonc/biosci670/blob/master/IntroductionComputationalMethods/05_IntroCompMethods_SolvingODEs.ipynb))


2. How could the authors test numerically whether the pattern that they discovered is indeed a feature of their ODE model?


3. After contacting the authors, you learned that their ODE is based on the *logistic growth model* 
$$\frac{dy}{dt}=r\,y\, (1-\frac{y}{K}) \, ,\tag{1}$$
with parameter choices $r=0.1$, $K=1$and initial value $y_0=0.1$.

  Approximate $y(t)$ numerically in the range from $t_0=0$ and $t_\text{max}=100$ using the explicit Euler method. Solve the model for 3 different step sizes $\Delta t = \{10, 5, 1\}$. What are the implications for question (1)?


4. (**optional**) As in question (3) but now use the Runge-Kutta RK4 method with fixed time step (see [notebook](https://github.com/cohmathonc/biosci670/blob/master/IntroductionComputationalMethods/05_IntroCompMethods_SolvingODEs.ipynb)) for solving the ODE.


5. (**optional**) Use the RK45 method with adaptive time stepping provided by `scipy.integrate` through the `solve_ivp` interface.
   The solution object contains time steps and estimated function values resulting from adaptive time stepping.


6. (**optional**) Read the description of the `rtol` option of `scipy.integrate.solve_ivp`. How do different choices of `rtol` to affect the size/number of time-steps computed by the adaptive algorithm. 
   



## Solution

### Possible explanations for oscillatory pattern

The oscillatory pattern could be a sign of numeric instability:
- Authors mention that explicit Euler method was used. This method requires small time steps to ensure convergence to the real solution and may become unstable otherwise.
- No precise information about the stepsize is provided, but the plot indicates that very large stepsizes $\Delta t > 10$ have been used for integration.

See example in exercise (2) of section *1.1.1 Euler Method (explicit)* in the [ODE notebook](https://github.com/cohmathonc/biosci670/blob/master/IntroductionComputationalMethods/05_IntroCompMethods_SolvingODEs.ipynb).


### Numeric tests to confirm results

The authors could repeat their simulation using a different integration method, and/or decrease the step size of their simulation.
If the oscillatory pattern cannot be reproduced with finer step sizes, it is likely an artifact of the numerical method for the specific ODE and parameters used for its integration.

### Solve with explicit Euler

In [ ]:
# from lecture notes L05 
# https://github.com/cohmathonc/biosci670/blob/master/IntroductionComputationalMethods/L05_IntroCompMethods_SolvingODEs.ipynb

def log_growth(t, y):
    return 0.1*y*(1-y)

def solve_euler(f, t, y_0):
    """
    Uses explicit euler method to solve ODE: y'=f(t, y) 
    with initial value y(t_0)=y_0.

    Args:
    - f: Function object that computes y', expected signature f(t, y), 
         where t is evaluation point and y is function value
    - t: array of evaluation points
    - y_0: initial value, i.e. y(t[0])

    Returns:
    - array containing approximated function values y(t[i]) 
    """
    y = np.zeros(t.shape)
    y[0] = y_0
    for i in range(0, len(t)-1):
        y[i+1] = y[i] + (t[i+1]-t[i]) * f(t[i],y[i])
    return y

In [ ]:
t_min = 0 
t_max = 100
y_0 = 0.1

fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(111) 

for delta_t in [10, 5, 1]:
    t = np.arange(t_min, t_max, delta_t)
    y_euler_exp = solve_euler(log_growth, t, y_0)
    # plot
    ax.plot(t,y_euler_exp, label='explicit Euler $\Delta t$ = %.2f'%delta_t)

ax.set_ylabel("normalized cell density N")
ax.set_xlabel('time')
ax.set_title('simulated cell density')
ax.legend()

Decreasing the stepsize results in a smooth non-oscillating curve that stabilizes at N=1 for large t.

Larger stepsizes result in oscillatory patterns around the carrying capacity $K$, $N=1$. This becomes more evident when extending the integration interval:

In [ ]:
t_min = 0 
t_max = 200
y_0 = 0.1

fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(111) 

for delta_t in [10, 15, 20, 25]:
    t = np.arange(t_min, t_max, delta_t)
    y_euler_exp = solve_euler(log_growth, t, y_0)
    # plot
    ax.plot(t,y_euler_exp, label='explicit Euler $\Delta t$ = %.2f'%delta_t)

ax.set_ylabel("normalized cell density N")
ax.set_xlabel('time')
ax.set_title('simulated cell density')
ax.legend()

The oscillatory pattern presented in the question is indeed a numeric artifact. 

However,  the oscillatory pattern shown in fig 1 cannot be reproduced with the parameters specified in Q3. This indicates that numerical stability depends on the ode and its parameterization as well as the integration stepsize (and also that different parameters have been used to produce the plot).

### RK4 (optional)

In [ ]:
# from lecture notes L05 
# https://github.com/cohmathonc/biosci670/blob/master/IntroductionComputationalMethods/L05_IntroCompMethods_SolvingODEs.ipynb

def solve_RK4(f, t, y_0):
    """
    Uses 4-step Runge-Kutta method to solve ODE: y'=f(t, y) 
    with initial value y(t_0)=y_0.

    Args:
    - f: Function object that computes y', expected signature f(t, y), 
         where t is evaluation point and y is function value
    - t: array of evaluation points
    - y_0: initial value, i.e. y(t[0])

    Returns:
    - array containing approximated function values y(t[i]) 
    """
    y = np.zeros(t.shape)
    y[0] = y_0
    for i in range(0, len(t)-1):
        delta_t = t[i+1]-t[i]
        k_1  = delta_t * f(t[i], y[i])
        k_2  = delta_t * f(t[i] + delta_t/2, y[i] + k_1/2)
        k_3  = delta_t * f(t[i] + delta_t/2, y[i] + k_2/2)
        k_4  = delta_t * f(t[i] + delta_t, y[i] + k_3)
        y[i+1] = y[i] + 1/6*(k_1 + 2*k_2 + 2*k_3 + k_4)
    return y



In [ ]:
t_min = 0 
t_max = 100
y_0 = 0.1

fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(111) 

for delta_t in [10, 5, 1]:
    t = np.arange(t_min, t_max, delta_t)
    y_euler_exp = solve_RK4(log_growth, t, y_0)
    ax.plot(t,y_euler_exp, label='RK4 $\Delta t$ = %.2f'%delta_t)

ax.set_ylabel("normalized cell density N")
ax.set_xlabel('time')
ax.set_title('simulated cell density')
ax.legend()

The approximation error of this method for a timestep of given size is smaller than the approximation error made by the Euler method.

This method is also less prone to the instabilities observed in Q3 for the Euler method.  

### RK45 from `scipy.integrate` (optional)

In [ ]:
# using RK45 solver via scipy.integrate.solve_ivp also shown in L05
# https://github.com/cohmathonc/biosci670/blob/master/IntroductionComputationalMethods/L05_IntroCompMethods_SolvingODEs.ipynb

from scipy.integrate import solve_ivp

t_min = 0 
t_max = 200
y_0 = 0.1

fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(111) 
sol = solve_ivp(log_growth, [t_min, t_max], [y_0], 
                method='RK45', rtol=1E-6)
# plot
ax.plot(sol.t,sol.y[0], label='RK45 (scipy.integrate)',
       marker='x')
ax.set_ylabel("normalized cell density N")
ax.set_xlabel('time')
ax.set_title('simulated cell density')
ax.legend()

###  Tolerance in adaptive time stepping (optional)

The `rtol` option allows to modify the maximum *relative error* that is considered acceptable in each timestep. Decreasing `rtol` to smaller values means that the solution algorithm tries to find more accurate approximations which will translate into smaller step sizes. In turn, increasing `rtol` means that less accurate solutions are acceptable, which implies that the stepsize can be increased.

###### About 
This notebook is part of the *biosci670* course on *Mathematical Modeling and Methods for Biomedical Science*.
See https://github.com/cohmathonc/biosci670 for more information and material.